##  Самойлович Константин Олегович
### Тестовое задание по курсу "Обработка естественного языка"

### Задание:
1. Используя язык программирования Python 3 необходимо выполнить предобработку текстовых данных:
- привести все слова к нижнему регистру;
- удалить пунктуацию;
- удалить нерелевантные слова (ссылки, слова на английском, и. т .д);
- удалить стоп слова с помощью готовых словарей ( при необходимости дополнить словарь);
- выполнить лемматизацию ( привести все слова в их начальную словоформу).
 
2. Отсортировать слова по частоте их употребления и выделить топ 100 слов.
 
3. В качестве результата предоставить код в формате ipynb или py, а также список полученных слов в текстовом документе.


In [ ]:
# Загрузка необходимых модулей и библиотек
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('all')
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Загрузка файла данных (лист исходного файла test_data.xlsx сохранен в формате csv)
from google.colab import files
uploaded = files.upload()

Saving test_data.xlsx - Лист1.csv to test_data.xlsx - Лист1.csv


In [ ]:
text = pd.read_csv('test_data.xlsx - Лист1.csv')
text.head(3)

NameError: ignored

In [ ]:
# Определяем стоп-слова
from nltk.corpus import stopwords
ru_stop_words = set(stopwords.words('russian'))
my_stop_words = ['т.е.', 'что-то', 'это', 'br', 'ваш', 'ваша', 'вашу', 'вашей', 'мы', 'мой',
                 'который', 'свой', 'быть', 'наш', 'тот', 'сам', 'г', 'твой', 'это']
ru_stop_words.update(my_stop_words) # к данному списку добавляем свои стоп-слова

# Определяем знаки пунктуации
import string
punctuations = list(string.punctuation)
my_stop_punkt = ['/', '«', '»', "''", '-', '_', '☝🏻ну', '.', '–', '—', '``', '...', '--']
punctuations.extend(my_stop_punkt)  # к данному списку добавляем свои знаки пунктуации

# Определяем цифры
digits = [str(i) for i in range(10)]

# Определяем эмодзи
!pip install emoji --upgrade
import emoji
emo = list(emoji.UNICODE_EMOJI['en'])

# Определяем англ.буквы
engl = list(string.ascii_letters)
# Cсылки и мусор
my_link = ['🏸', '/', '.']

# Библиотека для лемматизации текста
!pip install pymorphy2
import pymorphy2
m = pymorphy2.MorphAnalyzer()

In [ ]:
# ФУНКЦИЯ ОБРАБОТКИ ДАННЫХ
def text_norm(text, stop_words, punct, digits, emo, engl, link):
  text = text.replace ('.', ' ') # заменяем точку пробелами
  text = text.replace ('…', ' ') # заменяем многоточие пробелами
  tokens = word_tokenize(text.lower(), language = 'russian') # токенизируем и переводим в ниж. регистр
  tokens = [word for word in tokens if word not in stop_words] # Удаляем стопслова
  tokens = [word for word in tokens if word not in punct] # Удаляем знаки пунктуации
  tokens = [word for word in tokens if word[0] not in digits] # Удаляем цифры
  tokens = [word for word in tokens if word not in emo] # Удаляем эмодзи
  tokens = [word for word in tokens if word[0] not in engl]  # Удаляем англ.слова
  tokens = [word for word in tokens if word[0] not in link] # удаляем ссылки и мусор
  tokens = [m.parse(word)[0].normalized.word for word in tokens] # лемматизация текста
  new_text = ' '.join(tokens)
  return new_text

In [ ]:
func = lambda x: text_norm(x, ru_stop_words, punctuations, digits, emo, engl, my_link )


In [ ]:
text['norm_body'] = text['body'].apply(func) # генерируем колонку norm_body, прогоняя значения колонки body через функцию обработки

In [ ]:
text.head(1)

,url,author,type,body,norm_body
0,https://vk.com/wall-44933362_3142,id_woman_jj,Социальные сети,1 день = 01.01.2019 = закладываем январь 2019 ...,день закладывать январь год день реализация вн...


In [ ]:
# Подсчет количества повторений слов
def get_word_counts(texts):
    wordfreq = {}

    for sentence in texts:
        
        for word in word_tokenize(sentence):
            
            if word not in wordfreq:
                wordfreq[word] = 0 
            wordfreq[word] += 1
        
    return {k:v for k,v in sorted(wordfreq.items(), key=lambda kv: kv[1], reverse=True)}
    

In [ ]:
norm_texts = text['norm_body'].to_list()

word_counts = get_word_counts(norm_texts) # частота употребления слов, отсортированная по убыванию

In [ ]:
# word_counts - при запуске словарь покажет количество повторений слов
len(word_counts) # количество слов в словаре

23598

In [ ]:
# 100 наиболее повторяющихся слов (TOP100)
TOP100 = list(word_counts.keys())[0:99]

In [ ]:
TOP100 # некоторые слова (например, "который" и "свой") попали в этот список, хотя были внесены в перечень стоп-слов. 
       # Причину этого следует искать в процессе финишной лемматизаци.
       # При необходимости можно устранять дополнительными обработками.

NameError: ignored

In [ ]:
# Запись ТОП-100 слов в файл

f = open('C:\top100.txt', 'w') # Открыть файл для записи
# Цикл записи строк
for s in L:
    f.write(s + '\n') # записать каждую строку в отдельную строку файла
f.close() # Закрыть файл   